In [2]:
import numpy as np
from numpy.linalg import inv
from numpy.matlib import matrix  

np.set_printoptions(precision = 3, threshold = 10, edgeitems = 4, linewidth = 120)  

TRUNC_THRSHLD = 10**(-12)

def LPP(A: matrix, b: np.array, c: np.array, \
            _EPSILON : float) -> (int, np.array, float, np.array):
    """
    executes simplex method: phase 1 and 2
    @param A: constraint matrix
    @param b: independent terms in constraints
    @param c: costs vector
    """
    __m, __n = A.shape[0], A.shape[1]  # no. of rows, columns of A

    if __n < __m: raise ValueError("Incompatible dimensions "
                         "(no. of variables : {} > {} : no.of constraints".format(n, m))
    
    if b.shape != (__m,): raise \
        ValueError("Incompatible dimensions: c_j has shape {}, expected {}.".format(b.shape, (m,)))
    
    if c.shape != (__n,): raise \
        ValueError("Incompatible dimensions: c has shape {}, expected {}.".format(c.shape, (n,)))

    "Check full rank matrix"
    if not np.linalg.matrix_rank(A) == __m:
        # Remove ld rows:
        A = A[[i for i in range(__m) if not np.array_equal(np.linalg.qr(A)[1][i, :], np.zeros(__n))], :]
        __m = A.shape[0]  # Update no. of rows

    A[[i for i in range(m) if b[i] < 0]] *= -1  # Change sign of constraints
    b = np.abs(b)

    A_I = matrix(np.concatenate((A, np.identity(__m)), axis = 1))  # Phase 1 constraint matrix
    x_I = np.concatenate((np.zeros(__n), b))  # Phase 1 variable vector
    c_I = np.concatenate((np.zeros(__n), np.ones(__m)))  # Phase 1 c_j vector
    basic_I = set(range(__n, __n + __m))  # Phase 1 basic variable set

    print("Executing phase 1")
    ext_code_I, x_init, basic_init, z_I, _, ITERS_I = simplex_core(A_I, c_I, x_I, basic_I)
    # ^ Exit code, initial BFS, basis, z_I, and no. of iterations
    print("Phase I terminated.")

    assert ext_code_I == 0  # assert phase I has optimal solution
    if z_I > 0:
        print("\n")
        print_boxed("Unfeasible problem (z_I = {:.6g} > 0).".format(z_I))
        print("{} iterations in phase I.".format(ITERS_I), end = '\n\n')
        return 2, None, None, None
    
    # If artificial variable in the basis for the initial BFS, exit:
    if any(j not in range(__n) for j in basic_init): raise NotImplementedError("Artificial variables in basis")

    x_init = x_init[:__n]  # Get initial BFS for original problem

    print("Found initial BFS at x = \n{}.\n".format(x_init))

    """Phase II execution"""
    print("Executing phase 2")
    ext_code, x, basic, z, d, ITERS_II = simplex_core(A, c, x_init, basic_init)
    print("Phase II terminated.\n")

    if ext_code == 0:
        print_boxed("Found optimal solution at x =\n{}.\n\n".format(x) +
                    "Basic indexes: {}\n".format(basic) +
                    "Nonbasic indexes: {}\n\n".format(set(range(n)) - basic) +
                    "Optimal cost: {}.".format(z))
    
    elif ext_code == 1: print_boxed(
        "Unbounded problem. Found feasible direction d =\n{}\nfrom x =\n{}.".format(d, x)
    )

    print("{} iterations in phase I, {} iterations in phase II ({} total).".format(
        ITERS_I, ITERS_II, ITERS_I + ITERS_II), end = '\n\n')

    return ext_code, x, z, d

In [3]:
MAX_ITERS : int = 100000007 

def simplex_core(A: matrix, c: np.array, x: np.array, basic: set) \
        -> (int, np.array, set, float, np.array):
    """
    @param A: constraint matrix
    @param c: costs vector
    @param x: initial BFS
    @param basic: initial basic index set
    
    @return: a tuple consisting of the exit code, the value of x, basic index set,
    optimal cost, and BFS corresponding to feasible direction
    """
    __m, __n = A.shape[0], A.shape[1]  

    assert c.shape == (__n,) and x.shape == (__n,)
    
    assert isinstance(basic, set) and len(basic) == __m and \
           all(i in range(__n) for i in basic)  # Make sure that basic is a valid base

    B, N = list(basic), set(range(n)) - basic  # Basic /nonbasic index lists
    del basic
    B_inv = inv(A[:, B])  # Calculate inverse of basic matrix (`A[:, B]`)

    z = np.dot(c, x) # Objective function

    iteration : int = 1 
    while iteration <= MAX_ITERS: 
        r_q, q, p, theta, d = None, None, None, None, None
        print("\tIteration no. {}:".format(iteration), end='')

        """Optimality test"""
        prices = c[B] * B_inv  # Store product for efficiency

       # Bland rule
        optimum = True
        for q in N:  # Read in lexicographical index order
            r_q = np.ndarray.item(c[q] - prices * A[:, q])
            if r_q < 0:
                optimum = False
                break 
        
        # Minimal reduced cost rule
        #r_q, q = min([(np.ndarray.item(c[q] - prices * A[:, q]), q) \ 
        #              for q in N], key=(lambda tup: tup[0]))
        #optimum = (r_q >= 0)

        if optimum:
            print("\tfound optimum")
            return 0, x, set(B), z, None, iteration  # Found optimal solution


        """Feasible basic direction"""
        d = np.zeros(__n)
        for i in range(__m): d[B[i]] = trunc(np.ndarray.item(-B_inv[i, :] * A[:, q]))
        d[q] = 1


        """Max Step Length"""
        # List of tuples of "candidate" theta an corresponding index in basic list:
        neg = [(-x[B[i]] / d[B[i]], i) for i in range(__m) if d[B[i]] < 0]

        if len(neg) == 0:
            print("\tidentified unbounded problem")
            return 1, x, set(B),  None, d, iteration  # Flag problem as unbounded and return direction

        # Get theta and index (in basis) of exiting basic variable:
        theta, p = min(neg, key=(lambda tup: tup[0]))


        """Variable updates"""
        x = np.array([trunc(var) for var in (x + theta * d)])  # Update all variables
        assert x[B[p]] == 0

        z = trunc(z + theta * r_q)  # Update objective function value

        # Update inverse:
        for i in set(range(m)) - {p}:
            B_inv[i, :] -= d[B[i]]/d[B[p]] * B_inv[p, :]
        B_inv[p, :] /= -d[B[p]]

        N = N - {q} | {B[p]}  # Update nonbasic index set
        B[p] = q  # Update basic index list

        """Print status update"""
        print(
            "\tq = {:>2} \trq = {:>9.2f} \tB[p] = {:>2d} \ttheta* = {:>5.4f} \tz = {:<9.2f}"
                .format(q + 1, r_q, B[p] + 1, theta, z)
        )
        iteration += 1
    raise TimeoutError("Iterations maxed out (probably due to an endless loop)")

In [4]:
def print_boxed(msg: str) -> None:
    lines = msg.splitlines()
    max_len = max(len(line) for line in lines)

    if max_len > 100: raise ValueError("Overfull box")

    print('-' * (max_len + 4))
    for line in lines: print('| ' + line + ' ' * (max_len - len(line)) + ' |')
    print('-' * (max_len + 4))

def trunc(x: float) -> float: return x if abs(x) >= TRUNC_THRSHLD else 0

Executing phase 1
	Iteration no. 1:	q =  1 	rq =     -1.00 	B[p] =  1 	theta* = 3.0000 	z = 4.00     
	Iteration no. 2:	q =  2 	rq =     -2.50 	B[p] =  2 	theta* = 1.6000 	z = 0.00     
	Iteration no. 3:	found optimum
Phase I terminated.
Found initial BFS at x = 
[0.6 1.6 0.  0. ].

Executing phase 2
	Iteration no. 1:	found optimum
Phase II terminated.

---------------------------------
| Found optimal solution at x = |
| [0.6 1.6 0.  0. ].            |
|                               |
| Basic indexes: {0, 1}         |
| Nonbasic indexes: {2, 3}      |
|                               |
| Optimal cost: -5.4.           |
---------------------------------
3 iterations in phase I, 1 iterations in phase II (4 total).



(0, array([0.6, 1.6, 0. , 0. ]), -5.4, None)